In [17]:
# read file
with open('Q4/facebook_combined.txt', 'r') as file:
    edges = [tuple(map(int, line.strip().split())) for line in file]

In [18]:
nodes = set(sum(edges, ()))
members = len(nodes)
relations = len(edges)

print(f"Number of nodes: {members}")
print(f"Number of edges: {relations}")

Number of nodes: 4039
Number of edges: 88234


In [1]:
num_nodes = 8

In [20]:
import networkx as nx

graph = nx.Graph()
for s, d in edges:
    graph.add_edge(s, d)

In [3]:
import matplotlib.pyplot as plt
import random


def create_and_save_realizations_with_activation(graph, num_realization, output_directory):
    # Create realizations with edge activation and save them
    for i in range(num_realization):  # Create n realizations
        realization_graph = graph.copy()  # Create a copy of the original graph for each realization

        # Randomly activate edges with a 10% chance
        for edge in realization_graph.edges():
            if random.random() > 0.1:  # 10% chance of activation
                realization_graph.remove_edge(*edge)

        file_path = output_directory + f'/realization{i}.graphml'
        nx.write_graphml(realization_graph, file_path)

In [22]:
create_and_save_realizations_with_activation(graph, 10, "Q4")

In [2]:
import networkx as nx


def read_realizations(num_realization, directory):
    realization = []
    for i in range(num_realization):
        file_path = directory + f'/realization{i}.graphml'
        realization.append(nx.read_graphml(file_path))

    return realization

In [3]:
realizations = read_realizations(10, "Q4")
realizations

C)

In [4]:
def top_degree_nodes(graph, num_nodes):
    node_degrees = list(graph.degree)

    # Sort the nodes by degree in descending order
    sorted_nodes = sorted(node_degrees, key=lambda x: x[1], reverse=True)

    # Select the top n nodes and their degrees
    top_n_nodes = sorted_nodes[:num_nodes]

    return top_n_nodes

In [5]:
top_nodes = {}
for realization in realizations:
    s = top_degree_nodes(realization, num_nodes)
    for k in s:
        if k[0] not in top_nodes.keys():
            top_nodes[k[0]] = {}
            top_nodes[k[0]]['sum'] = k[1]
            top_nodes[k[0]]['num'] = 1
        else:
            top_nodes[k[0]]['sum'] += k[1]
            top_nodes[k[0]]['num'] += 1

In [6]:
top_nodes_list = []

for k in top_nodes.keys():
    top_nodes_list.append((k, top_nodes[k]['sum'] / top_nodes[k]['num']))

degree_nodes = sorted(top_nodes_list, key=lambda x: x[1], reverse=True)[:num_nodes]
degree_nodes

[('107', 106.8),
 ('1684', 82.3),
 ('1912', 76.9),
 ('3437', 56.1),
 ('0', 35.5),
 ('2604', 34.0),
 ('1730', 34.0),
 ('2586', 33.0)]

In [7]:
list(zip(*degree_nodes))[0]

('107', '1684', '1912', '3437', '0', '2604', '1730', '2586')

In [8]:
def marginal_gain(graph, nodes):
    # Initialize a set to keep track of reachable nodes
    reachable_nodes = set()

    # Use Breadth-First Search (BFS) to find reachable nodes from the seed set
    for node in nodes:
        reachable_nodes.update(nx.bfs_tree(graph, source=node).nodes())

    return len(reachable_nodes)


In [9]:
import time

start_time = time.time()

top_nodes_list = []

top_nodes = {}
for realization in realizations:
    s = top_degree_nodes(realization, num_nodes)
    for k in s:
        if k[0] not in top_nodes.keys():
            top_nodes[k[0]] = {}
            top_nodes[k[0]]['sum'] = k[1]
            top_nodes[k[0]]['num'] = 1
        else:
            top_nodes[k[0]]['sum'] += k[1]
            top_nodes[k[0]]['num'] += 1


for k in top_nodes.keys():
    top_nodes_list.append((k, top_nodes[k]['sum'] / top_nodes[k]['num']))

degree_nodes = sorted(top_nodes_list, key=lambda x: x[1], reverse=True)[:num_nodes]


top_node_gain = 0

for realization in realizations:
    top_node_gain += marginal_gain(realization, list(zip(*degree_nodes))[0])

top_node_gain /= 10

end_time = time.time()
elapsed_time = end_time - start_time
print("Selected Nodes: ",list(zip(*degree_nodes))[0])
print("Elapsed time: ", elapsed_time)
print("f(S): ",top_node_gain)

Selected Nodes:  ('107', '1684', '1912', '3437', '0', '2604', '1730', '2586')
Elapsed time:  2.6521644592285156
f(S):  2988.5


In [10]:
import random


def get_random_nodes(graph, num_nodes):
    return random.sample(graph.nodes(), num_nodes)

In [16]:
import time

start_time = time.time()

random_nodes = []

for realization in realizations:
    random_nodes.extend(get_random_nodes(realization, num_nodes))

random_nodes = random.sample(random_nodes, num_nodes)


random_node_gain = 0

for realization in realizations:
    random_node_gain += marginal_gain(realization, random_nodes)

random_node_gain /= 10

end_time = time.time()
elapsed_time = end_time - start_time
print("Selected Nodes: ",random_nodes)
print("Elapsed time: ", elapsed_time)
print("f(S): ",random_node_gain)

Selected Nodes:  ['2202', '3347', '1580', '365', '3891', '1905', '187', '3807']
Elapsed time:  1.667137622833252
f(S):  2975.3


D)

In [17]:
import numpy as np


def greedy(realizations):
    """
    Find k nodes with the largest spread (determined by IC) from a igraph graph
    using the Greedy Algorithm.
    """

    spreads = []
    solution = []

    for _ in range(num_nodes):
        best_node = -1
        best_spread = -np.inf


        nodes = set(realizations[0].nodes()) - set(solution)
        for node in nodes:
            gain_node = 0
            for realization in realizations:
                gain_node += marginal_gain(realization, solution + [node])
            gain_node /= 10

            if gain_node > best_spread:
                best_spread = gain_node
                best_node = node

        solution.append(best_node)
        spreads.append(best_spread)

    return solution, spreads



In [18]:
import time

start_time = time.time()

solution, spreads = greedy(realizations)

end_time = time.time()
elapsed_time = end_time - start_time

print("Selected Nodes: ",solution)
print("Elapsed time: ", elapsed_time)
print("f(S): ",spreads)

Selected Nodes:  ['2758', '343', '776', '3980', '4030', '3549', '3382', '3918']
Elapsed time:  21425.70372915268
f(S):  [2933.5, 2990.7, 3041.6, 3055.2, 3059.4, 3063.5, 3067.5, 3071.4]


In [19]:
import heapq

def celf(realizations):
    """
    Find k nodes with the largest spread (determined by IC) from a igraph graph
    using the Cost Effective Lazy Forward Algorithm, a.k.a Lazy Greedy Algorithm.
    """

    gains = []
    nodes = set(realizations[0].nodes())
    for node in nodes:
        s = 0
        for realization in realizations:
            s += marginal_gain(realization, node)
        spread = s / 10
        heapq.heappush(gains, (-spread, node))


    spread, node = heapq.heappop(gains)
    solution = [node]
    spread = -spread
    spreads = [spread]

    for _ in range(num_nodes - 1):
        matched = False

        while not matched:


            _, current_node = heapq.heappop(gains)

            s = 0
            for realization in realizations:
                s += marginal_gain(realization, solution + [current_node])

            spread_gain = (s / 10) - spread

            heapq.heappush(gains, (-spread_gain, current_node))
            matched = gains[0][1] == current_node

        spread_gain, node = heapq.heappop(gains)
        spread -= spread_gain
        solution.append(node)
        spreads.append(spread)

    return solution, spreads


In [20]:
import time

start_time = time.time()


solution, spreads = celf(realizations)

end_time = time.time()
elapsed_time = end_time - start_time

print("Selected Nodes: ",solution)
print("Elapsed time: ", elapsed_time)
print("f(S): ",spreads)

Selected Nodes:  ['2058', '343', '776', '3980', '4030', '3549', '3382', '3579']
Elapsed time:  4340.257801055908
f(S):  [2420.9, 2990.7, 3041.6, 3055.2, 3059.4, 3063.5, 3067.5, 3071.4]
